<a href="https://colab.research.google.com/github/Nouf-Alabbasi/oKUmura_AI_Telecom_challenge/blob/main/No_router.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Test_3
run all cells with ⚫
for each query run the cells with 🔴

##⚫ install req

In [1]:
!pip install streamlit
!pip install langchain
!pip install langchain langchain-community chromadb jq
!pip install openAI
!pip install langchain-openai
!pip install faiss-gpu
!pip install python-docx
!pip install unstructured
!pip install --quiet langchain_experimental langchain_openai
!pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.6/657.6 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

##⚫ setup

In [2]:
import streamlit as st
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
import os
from google.colab import userdata
import openai
from docx import Document
from langchain_community.document_loaders import UnstructuredWordDocumentLoader

import json
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.vectorstores.faiss import FAISS
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm

In [3]:
# Set your OpenAI API key
openai.api_key = userdata.get('OpenAI')
os.environ["OPENAI_API_KEY"] = userdata.get('OpenAI')



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#query

In [ ]:
query = "What message does the AMF send to the NG-RAN during mobility registration, AM policy modification, and handover procedures? [3GPP Release 18]"

##chunking

###header aware chunking
https://saeedesmaili.com/text-chunking-headings-grouping-parsing-documents-with-pandoc/

In [2]:
# !apt-get install pandoc
!pip install sh

In [3]:
from sh import pandoc

In [4]:
import re

def identify_doc_type(doc):
    '''
    categorizes a plaintext doc based on the format of the toc.
    '''
    if re.search(r'.*\n\n\n-\s{3}.*', doc):
        return "TOC_WITH_TITLE"
    elif re.search(r'-\s{3}.*\n\n.*', doc):
        return "TOC_WITHOUT_TITLE"
    else:
        return "NO_TOC_TITLE"


In [5]:
def read_doc(path):
    '''
    reads a text file and returns toc and full text.
    '''
    doc = str(pandoc(path, "-t", "plain", "--toc", "--standalone"))
    doc_type = identify_doc_type(doc)

    if doc_type == "TOC_WITH_TITLE":
        doc = re.sub('.*\n\n\n-', '-', doc)
        toc, text = doc.split('\n\n', 1)
    elif doc_type == "TOC_WITHOUT_TITLE":
        toc, text = doc.split('\n\n', 1)
    else:
        toc, text = "", doc

    return toc, text

In [6]:
def cleanup_plaintext(text):
    '''
    gets the full text of a document and returns cleaned-up text.
    '''
    # Remove images
    text = text.replace("[image]", "")
    text = text.replace("[]", "")

    # Replace single \n with space (if the next char is not \n or -)
    text = re.sub('(?<!\n)\n(?!(\n|-))', ' ', text)

    # Replace any sequence of two or more newlines with \n\n
    text = re.sub('\n{2,}', '\n\n', text)

    # Replace multiple spaces with single space
    text = re.sub('(?<!\n) +', ' ', text)
    return text

In [7]:
def split_text(toc, text):
    '''
    gets the toc and cleaned text, and returns chunks of texts:
    ["Heading [SEP] Text", ]
    '''
    headings = [line.strip('- \n') for line in toc.split('\n')]
    paragraphs = text.split("\n\n")

    current_heading = ""
    list_group = ""
    text_chunks = []
    for para in paragraphs:

        # use the new heading if we've moved to a new heading section
        if len(headings) > 0 and para == headings[0]:
            current_heading = headings[0]
            headings.pop(0)
            continue

        # group bullet points as a single chunk of text
        if para.startswith("- "):
            list_group += para + " "
            continue
        elif list_group != "":
            para = list_group
            list_group = ""

        # if we're at the beginning of a document and
        # we haven't seen any headings yet
        if current_heading == "":
            text_chunks.append(para.strip())
        else:
            text_chunks.append(f"{current_heading} [SEP] {para}".strip())

    return text_chunks

In [21]:
path_to_file = "/content/drive/MyDrive/KU/RAG_telco_files/documents/"

In [20]:
import os
import pandas as pd

df = pd.DataFrame()
allowed_filetypes = ['.docx']


file_list = os.listdir(path_to_file)
for file in file_list:
  filename, filetype = os.path.splitext(file)
  if filetype in allowed_filetypes:
      full_path = os.path.join(directory, file)
      print(f" Filename: {filename}")
      toc, text = read_doc(full_path)
      text_cleaned = cleanup_plaintext(text)
      text_chunks = split_text(toc, text_cleaned)
      print(f" text chunk: {text_chunks[4]}")
      df_new = pd.DataFrame(text_chunks, columns=["text"])
      df_new[["directory", "filename", "filetype"]] = directory, filename, filetype
      df = pd.concat([df, df_new])

df.reset_index(drop=True, inplace=True)

<generator object _walk at 0x7cd45abbc820>


In [16]:
df

""


In [15]:
# doc = str(pandoc(path_to_file, "-t", "plain", "--toc", "--standalone"))
# doc_type = identify_doc_type(doc)

In [16]:
doc_type

'TOC_WITHOUT_TITLE'

In [17]:
doc

'-   \n    Contents\n-   \n    Foreword\n-   Introduction\n-   \n    1 Scope\n-   2 References\n-   3 Definitions of terms, symbols and abbreviations\n    -   3.1 Terms\n    -   3.2 Symbols\n    -   3.3 Abbreviations\n-   4 Overview\n    -   4.1 General\n    -   4.2 Service provisioning\n    -   4.3 Registration\n    -   4.4 EAS discovery\n    -   4.5 Capability exposure to EAS and EEC\n    -   4.6 Support for service continuity\n    -   4.7 Security\n    -   4.8 Dynamic EAS instantiation triggering\n    -   4.9 Charging\n-   5 Architectural requirements\n    -   5.1 General\n    -   5.2 Architectural requirements\n        -   5.2.1 General requirements\n        -   5.2.2 Edge configuration data\n        -   5.2.3 Registration\n        -   5.2.4 EAS discovery\n        -   5.2.5 Capability exposure to EASs\n        -   5.2.6 Security\n        -   5.2.7 Subscription service\n        -   5.2.8 Traffic management\n        -   5.2.9 Lifecycle management\n        -   5.2.10 Edge application 

##semantic chunking

create text splitter

In [5]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

text_splitter = SemanticChunker(OpenAIEmbeddings())

In [23]:
path_to_file = "/content/drive/MyDrive/KU/RAG_telco_files/documents/23558-i50.docx"
loader = UnstructuredWordDocumentLoader(path_to_file, mode="elements")
data = loader.load ()

In [26]:
text = ""
for d in data:
  # print(d.page_content)
  text = text + " " + d.page_content

In [1]:
# text

In [27]:
docs = text_splitter.create_documents([text])
print(docs[0].page_content)

 3GPP TS 23.558 V18.5.0 (2023-12) Technical Specification 3rd Generation Partnership Project; Technical Specification Group Services and System Aspects; Architecture for enabling Edge Applications;  (Release 18) The present document has been developed within the 3rd Generation Partnership Project (3GPP TM) and may be further elaborated for the purposes of 3GPP. The present document has not been subject to any approval process by the 3GPP Organizational Partners and shall not be implemented.


In [31]:
embeddings = OpenAIEmbeddings()

db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever()


In [34]:
# docs_ret = retriever.invoke("Which 3GPP specification defines the architecture, procedures, and information flows to enable edge applications over 3GPP networks?")
# docs_ret = retriever.invoke("Which 3GPP specification provides the service requirements for V2X services? [3GPP Release 14]")
# docs_ret = retriever.invoke("What functionalities does the Edge Application Server Discovery Function (EASDF) include? [3GPP Release 18]")
docs_ret = retriever.invoke("What is the purpose of the Edge Configuration Server (ECS) specified in TS 23.558?")

In [35]:
docs_ret

[Document(page_content=' 3GPP TS 23.558 V18.5.0 (2023-12) Technical Specification 3rd Generation Partnership Project; Technical Specification Group Services and System Aspects; Architecture for enabling Edge Applications;  (Release 18) The present document has been developed within the 3rd Generation Partnership Project (3GPP TM) and may be further elaborated for the purposes of 3GPP. The present document has not been subject to any approval process by the 3GPP Organizational Partners and shall not be implemented.'),
 Document(page_content='6.3.8\tNotification management server Notification management server is as specified in 3GPP\xa0TS\xa023.434\xa0[13]. 6.3.9\tCloud Enabler Server (CES) CES provides supporting functions needed for CASs. The CES is part of the EEL and it does not have service area restriction. CES facilitates service continuity between EAS and CAS. Functionalities of the CES are: a)\tproviding access to network capability information (e.g. location information) to th

In [ ]:
# from llama_index.core import SimpleDirectoryReader

# from llama_index.core.node_parser import (
#     SentenceSplitter,
#     SemanticSplitterNodeParser,
# )
# from llama_index.embeddings.openai import OpenAIEmbedding

# def chunk_doc(file_path):
#     documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
#     embed_model = OpenAIEmbedding()
#     splitter = SemanticSplitterNodeParser(
#         buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
#     )

#     # also baseline splitter
#     base_splitter = SentenceSplitter(chunk_size=512)
#     nodes = splitter.get_nodes_from_documents(documents)
#     filename = os.path.basename(file_path)

#     chunks = [{"text": node.get_content(), "source": filename} for node in nodes]
#     print(chunks)
#     return chunks

# def process_all_documents(folder_path):
#     all_chunks = []
#     for filename in os.listdir(folder_path):
#         if filename.endswith(".docx"):
#             file_path = os.path.join(folder_path, filename)
#             chunks = chunk_doc(file_path)
#             all_chunks.extend(chunks)
#     return all_chunks

# # Set the folder path to your "Documents" directory
# folder_path = "/content/3GPP-Release18/Documents"
# all_document_chunks = process_all_documents(folder_path)

# # Example: Print the first chunk of each document
# for chunk in all_document_chunks:
#     print(chunk)

# Document_ds = [chunk_doc('/content/3GPP-Release18/Documents/' + doc['id']) for doc in document_ds]

ModuleNotFoundError: No module named 'llama_index'

##⚫ recursive chunking

In [ ]:

# Cache the function to load and process PDF documents
@st.cache(allow_output_mutation=True)

def load_and_process_docx(docx_folder_path):
  documents = []
  splits = []
  file_list = os.listdir(docx_folder_path)
  for file in file_list:
    if file.endswith(".docx"):
      print(file)
  for file in tqdm(file_list[:3]):
    if file.endswith(".docx"):
      loader = UnstructuredWordDocumentLoader(docx_folder_path+file)
      data = loader.load()
      documents.append(data)

  text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200, separators=["\n\n", "\n", "", " ", ""])

  for doc in documents:
    splits.extend(text_splitter.split_documents(doc))
  # docx_loader = DirectoryLoader(folder_path, glob="./*.docx", loader_cls=UnstructuredWordDocumentLoader)
  # documents += docx_loader.load()
  # text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
  # texts = text_splitter.split_documents(documents)

  return splits

# Cache the function to initialize the vector store with documents
@st.cache(allow_output_mutation=True)
def initialize_vectorstore(_splits):
    return FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=userdata.get('OpenAI')))

2024-06-27 10:26:12.968 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new caching commands, `st.cache_data` or `st.cache_resource`.
More information [in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching logic used by `st.cache_data` and `st.cache_resource`.
This might lead to some problems or unexpected behavior in certain edge cases.

2024-06-27 10:26:13.175 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new caching commands, `st.cache_data` or `st.cache_resource`.
More information [in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching logic used by `st.cache_data` and `st.cache_resource`.
This might lead to some problems or unexpected behavior in certain edge cases.



In [ ]:
folder_path = "/content/drive/MyDrive/KU/RAG_telco_files/documents/"

In [ ]:
splits = load_and_process_docx(folder_path)
# vectorstore = initialize_vectorstore(splits)
# embeddings = OpenAIEmbeddings()

# db = FAISS.from_documents(splits, embeddings)

# db.save_local("/content/drive/MyDrive/KU/RAG_telco_files/faiss_index_v3")

# # db = FAISS.load_local("/content/drive/MyDrive/KU/RAG_telco_files/faiss_index_v2", embeddings, allow_dangerous_deserialization=True)

# # # to load the db
# # db.load(faiss_file_path="/content/drive/MyDrive/KU/RAG_telco_files/", sql_url= "sqlite:///")

ModuleNotFoundError: No module named 'lib'

In [ ]:
for s in splits:
  print(s)

page_content="3GPP TS 29.591 V18.4.0 (2023-12)\n\nTechnical Specification\n\n3rd Generation Partnership Project;\n\nTechnical Specification Group Core Network and Terminals;\n\n5G System; Network Exposure Function Southbound Services;\n\nStage 3\n\n(Release 18)\n\nThe present document has been developed within the 3rd Generation Partnership Project (3GPP TM) and may be further elaborated for the purposes of 3GPP..\nThe present document has not been subject to any approval process by the 3GPP Organizational Partners and shall not be implemented.\nThis Specification is provided for future development work within 3GPP only. The Organizational Partners accept no liability for any use of this Specification.\nSpecifications and Reports for implementation of the 3GPP TM system should be obtained via the 3GPP Organizational Partners' Publications Offices.\n\n3GPP TS 29.591 V18.4.0 (2023-12)\n4\nRelease 18\n\n\n\n3GPP\n\nPostal address\n\n3GPP support office address\n\n650 Route des Lucioles - 

##⚫ 🔴 retrieval

In [ ]:
retriever = db.as_retriever()

In [ ]:
docs = retriever.invoke(query)

In [ ]:

# for d in docs:
#   Retrievals.append(d.page_content)

# Retrievals
filename = docs[0].metadata['source'].split('/')[-1]
context = docs[0].page_content

In [ ]:
len(docs)

4

##⚫ generator

##LLM(*Phi*-2)

In [ ]:
!pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

In [ ]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True).to('cuda')
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)


RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def phi2(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)
    # outputs = model.generate(**inputs, max_length=2048, max_new_tokens=50,)
    outputs = model.generate(**inputs, max_new_tokens=50, repetition_penalty=1.15)#, do_sample=True, top_p=0.90, num_return_sequences=3)
    text = tokenizer.batch_decode(outputs)[0]

    return text

prompt = '''def print_prime(n):
   """
   Print all primes between 1 and n
   """'''

phi2(prompt)

##generate

In [ ]:
content = ""
# docs.pop()
# docs.pop()
# for index, elem in enumerate(docs):
#   i = index+1
#   content = content + "\n\nretrieval \n" + elem.page_content
#   print(elem.metadata['source'].split('/')[-1])

# o = ["option 1 EAS address information and DNN", "option 2 EAS address information and S-NSSAI", "option 3 EAS address information, DNN, S-NSSAI, and AF Identifier", "option 4 EAS address information only"]
options = """
option 1: 5G access stratum time distribution indication
option 2: Uu time synchronization error budget
option 3: Clock quality detail level
option 4: UE Context Modification Request
option 5: N2 connection"""

# options = "\n".join(o)


syst_prompt = f"""
Instruct:
Please provide the answers to the following multiple choice question.
{query}

Considering the following context:
{content}

Please provide the answers to the following multiple choice question.
{query}
The output should be in the format: Option <Option id>

Options:
Write only the option number corresponding to the correct answer:\n{options}

Output:
"""

In [ ]:
# for i in docs:
#   print(i)
syst_prompt

In [ ]:
predicted_answers_str = phi2(syst_prompt)
print(f"predicted_answers_str {predicted_answers_str}")
predicted_answers_str = predicted_answers_str.split("Output:")[-1]
print(predicted_answers_str)

In [ ]:
predicted_answers_str = phi2(syst_prompt)
print(f"predicted_answers_str {predicted_answers_str}")
predicted_answers_str = predicted_answers_str.split("Output:")[-1]
print(predicted_answers_str)

In [ ]:
output = """
<option id="4">UE Context Modification Request</option>
<|endoftext|>

<option id="4">UE Context Modification Request</option>
<|endoftext|>"""

prompt = f"""Given the text below, the first option number that appears
give you output in the form: <option #>, where # is a placeholder for the option number
asnwer:
'{output}'

Output:
"""
predicted_answers_str = phi2(prompt)
print(f"predicted_answers_str {predicted_answers_str}")
predicted_answers_str = predicted_answers_str.split("Output:")[-1]
print(predicted_answers_str)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=50) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


predicted_answers_str Given the text below, the first option number that appears
give you output in the form: <option #>, where # is a placeholder for the option number
asnwer:
'
<option id="4">UE Context Modification Request</option>
<|endoftext|>

<option id="4">UE Context Modification Request</option>
<|endoftext|>'

Output:
```
The total cost of all items purchased by John is $45.00
```


### Subsections:
1. Introduction to `format_map()` method
2. Using `format_map()` with diction

```
The total cost of all items purchased by John is $45.00
```


### Subsections:
1. Introduction to `format_map()` method
2. Using `format_map()` with diction


In [ ]:
prompt

'Given the text below, the first option number that appears\ngive you output in the form: <option #>, where # is a placeholder for the option number\nasnwer:\nOutput:\n<option id="4">UE Context Modification Request</option>\n<|endoftext|>\n\n<option id="4">UE Context Modification Request</option>\n<|endoftext|>\n\nOutput:\n'

#test on test dataset

In [ ]:
import csv
path = '/content/drive/MyDrive/KU/RAG_telco_files'

In [ ]:
with open(path+"/Data/TeleQnA_testing1.json", "r") as file:
  test_questions= json.load(file)

questions_list=[]
for ind, q in test_questions.items():
  index = ind.split()[1]
  if (int(index) > 8361):
    questions_list.append(ind)


# ==============
# questions_list = [ 8153, 8165, 8210, 8216, 8223, 8231, 8266, 8272, 8301, 8310, 8314, 8318, 8342, 8347, 8350, 8362, 8363, 8371, 8403 ]

# # None of the above + none of the above questions
# # questions_list = ['question 8138', 'question 8186', 'question 8285', 'question 8301', 'question 8401', 'question 8508','question 8568', 'question 8677', 'question 8703', 'question 8779', 'question 8965', 'question 9213', 'question 9280', 'question 9571', 'question 9585', 'question 9626', 'question 9777', 'question 8186', 'question 8285', 'question 8350', 'question 8362', 'question 8429','question 8508','question 8677', 'question 8716','question 8779','question 8814','question 8868','question 8886','question 9167','question 9213','question 9319','question 9355','question 9394','question 9434','question 9585','question 9626','question 9738','question 9741','question 9767','question 9962']
# print(questions_list)
# ==============

with open(path+"/TeleQnA.json", "r") as file:
  questions= json.load(file)

with open(path+"/TeleQnA.json", "r") as file:
  questions= json.load(file)

# NOTE: testing doesn't have answers

# split data
  # tbd


fields = ['Task','Question_ID','question', 'category', 'category_pred', 'Answer_ID_RAG',"Prompt",'full_output','no context', 'actual answer',"match?"]
off_fields = ['Question_ID', 'Answer_ID', 'Task']
filename = path+"/results/_ALL_with_cat_results_no_candidate_answers_openAI_embed"


results_file = open(filename+".csv", 'w')
writer = csv.writer(results_file)
writer.writerow(fields)

off_results_file = open(filename+"_official.csv", 'w')
off_writer = csv.writer(off_results_file)
off_writer.writerow(off_fields)

retriever = db.as_retriever()
print(questions_list)
try:
    for ind, q in tqdm(questions.items(), desc="Answering Questions"):
      index = int(ind.split()[1])
      if ind in questions_list:

          # get all options
          options =[]
          options_dict = dict(filter(lambda item: "option" in item[0], q.items()))
          for opt in options_dict:
            last_opt = ""
            options.append(f"{opt}: {options_dict[opt]}")

          category = "none"

          # get context
          docs = retriever.invoke(q["question"])
          content = ""
          for index, elem in enumerate(docs):
            i = index+1
            content = content + "\n\nretrieval \n" + elem.page_content
            print(elem.metadata['source'].split('/')[-1])

          syst_prompt = f"""
          Instruct:
          Please provide the answers to the following multiple choice question.
          {query}

          Considering the following context:
          {content}

          Please provide the answers to the following multiple choice question.
          {query}
          The output should be in the format: Option <Option id>

          Options:
          Write only the option number corresponding to the correct answer:\n{options}

          Output:
          """

          syst_prompt_no_context = f"""
          Instruct:
          Please provide the answers to the following multiple choice question.
          {query}

          Considering the following context:


          Please provide the answers to the following multiple choice question.
          {query}
          The output should be in the format: Option <Option id>

          Options:
          Write only the option number corresponding to the correct answer:\n{options}

          Output:
          """

          predicted_answers_str = phi2(syst_prompt)
          predicted_answers_no_context = phi2(syst_prompt)
          print(f"predicted_answers_str {predicted_answers_str}")
          print(f"predicted_answers_no_context {predicted_answers_no_context}")
          response = predicted_answers_str.split("Output:")[-1]
          print(predicted_answers_str)

          # save to csv
          # fields = ['Task','Question_ID','question', 'category', 'category_pred', 'Answer_ID_RAG',"Prompt",'full_output',' no context' 'actual answer',"match?"]

          if (response is not None):
            row = ["phi_2", ind, q["question"],q["category"],category, "None",response,syst_prompt,predicted_answers_no_context,q["answer"],""]
            off_row = [ind, response[0], "phi_2"]
          else:
            row = ["phi_2", ind, q["question"],q["category"],category, "None","None","NONE", q["answer"],"None","None"]
            off_row = [ind, "None", "phi_2"]
          writer.writerow(row)
          off_writer
      # else:
      #   print(f"{ind} not in list")

finally:
    results_file.close()
    off_results_file.close()


['question 8362', 'question 8363', 'question 8364', 'question 8369', 'question 8371', 'question 8381', 'question 8389', 'question 8396', 'question 8401', 'question 8403', 'question 8405', 'question 8411', 'question 8417', 'question 8426', 'question 8429', 'question 8434', 'question 8437', 'question 8445', 'question 8455', 'question 8456', 'question 8460', 'question 8462', 'question 8476', 'question 8485', 'question 8493', 'question 8499', 'question 8503', 'question 8508', 'question 8523', 'question 8525', 'question 8531', 'question 8538', 'question 8541', 'question 8546', 'question 8552', 'question 8568', 'question 8570', 'question 8576', 'question 8578', 'question 8579', 'question 8598', 'question 8600', 'question 8601', 'question 8611', 'question 8614', 'question 8617', 'question 8620', 'question 8627', 'question 8635', 'question 8636', 'question 8638', 'question 8639', 'question 8641', 'question 8650', 'question 8654', 'question 8655', 'question 8657', 'question 8662', 'question 866

Answering Questions:  81%|████████  | 8079/10000 [00:00<00:00, 80783.53it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Answering Questions:  84%|████████▎ | 8362/10000 [00:00<00:00, 9930.10it/s] 

28838-i00.docx
28838-i00.docx
rel_15.docx
33117-i20.docx


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

#drafts

##Test RAG with no router + using langchain from [link](https://github.com/rasyosef/rag-with-phi-2-and-langchain/blob/main/app.py)

###install req

In [ ]:
!pip install gradio
!pip install langchain
!pip install langchain-community
!pip install transformers
!pip install torch
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 619.0 kB/s 

In [ ]:
!pip install accelerate
# !pip -qqq install bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 1.9 MB/s eta 0:00:00


In [ ]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 9.0 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=a95d529ed6c3fc817148ecd45741ca98c008c08005aad65ba43ca50b1396892f
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install transformers torch langchain langchain-community


###setup

In [ ]:
# import gradio as gr

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.vectorstores.faiss import FAISS
from langchain.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain.chains import RetrievalQA
from langchain.prompts.prompt import PromptTemplate
from langchain.vectorstores.base import VectorStoreRetriever

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

from transformers import TextIteratorStreamer
from threading import Thread

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

##test 1

In [ ]:

# Prompt template
template = """Instruction:
You are an AI assistant for answering questions about the provided context.
You are given the following extracted parts of a long document and a question. Provide a detailed answer.
If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
=======
{context}
=======
Question: {question}
Output:\n"""

QA_PROMPT = PromptTemplate(template=template, input_variables=["question", "context"])

# Load Phi-2 model from hugging face hub
model_id = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.float32, device_map="auto", trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# sentence transformers to be used in vector store
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/msmarco-distilbert-base-v4",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": False},
)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/319 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Returns a faiss vector store retriever given a txt file
def prepare_vector_store_retriever(filename):
    # Load data
    loader = UnstructuredFileLoader(filename)
    raw_documents = loader.load()

    # Split the text
    # text_splitter = CharacterTextSplitter(
    #     separator="\n\n", chunk_size=800, chunk_overlap=0, length_function=len
    # )
    text_splitter = RecursiveCharacterTextSplitter( chunk_size=800, chunk_overlap=0, separators=[" ", ",", "\n"])

    documents = text_splitter.split_documents(raw_documents)

    # Creating a vectorstore
    vectorstore = FAISS.from_documents(
        documents, embeddings, distance_strategy=DistanceStrategy.DOT_PRODUCT
    )

    return VectorStoreRetriever(vectorstore=vectorstore, search_kwargs={"k": 2})


# Retrieveal QA chian
def get_retrieval_qa_chain(text_file, hf_model):
    retriever = default_retriever
    if text_file != default_text_file:
        retriever = prepare_vector_store_retriever(text_file)

    chain = RetrievalQA.from_chain_type(
        llm=hf_model,
        retriever=retriever,
        chain_type_kwargs={"prompt": QA_PROMPT},
    )
    return chain


# Generates response using the question answering chain defined earlier
def generate(question, answer, text_file, max_new_tokens):
    streamer = TextIteratorStreamer(
        tokenizer=tokenizer, skip_prompt=True, skip_special_tokens=True, timeout=300.0
    )
    phi2_pipeline = pipeline(
        "text-generation",
        tokenizer=tokenizer,
        model=model,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        device_map="auto",
        streamer=streamer,
    )

    hf_model = HuggingFacePipeline(pipeline=phi2_pipeline)
    qa_chain = get_retrieval_qa_chain(text_file, hf_model)

    query = f"{question}"

    if len(tokenizer.tokenize(query)) >= 512:
        query = "Repeat 'Your question is too long!'"

    thread = Thread(target=qa_chain.invoke, kwargs={"input": {"query": query}})
    thread.start()

    response = ""
    for token in streamer:
        response += token
        yield response.strip()


# replaces the retreiver in the question answering chain whenever a new file is uploaded
def upload_file(file):
    return file, file

In [ ]:
default_text_file = "/content/23288-i40.txt"
default_retriever = prepare_vector_store_retriever(default_text_file)

In [ ]:
text_file = default_text_file
ques = "What is the role of MDA MnS producer in the management function? [3GPP Release 17]"

# e(question, answer, text_file, max_new_tokens):
generate(ques,"ans",text_file,64)

<generator object generate at 0x7dd6178e5850>

In [ ]:

with gr.Blocks() as demo:
  #   gr.Markdown(
  #       """
  # # Retrieval Augmented Generation with Phi-2: Question Answering demo
  # ### This demo uses the Phi-2 language model and Retrieval Augmented Generation (RAG). It allows you to upload a txt file and ask the model questions related to the content of that file.
  # ### If you don't have one, there is a txt file already loaded, the new Oppenheimer movie's entire wikipedia page. The movie came out very recently in July, 2023, so the Phi-2 model is not aware of it.
  # The context size of the Phi-2 model is 2048 tokens, so even this medium size wikipedia page (11.5k tokens) does not fit in the context window.
  # Retrieval Augmented Generation (RAG) enables us to retrieve just the few small chunks of the document that are relevant to the our query and inject it into our prompt.
  # The model is then able to answer questions by incorporating knowledge from the newly provided document. RAG can be used with thousands of documents, but this demo is limited to just one txt file.
  # """
  #   )

    default_text_file = "/content/23288-i40.txt"
    default_retriever = prepare_vector_store_retriever(default_text_file)

    text_file = gr.State(default_text_file)

    # gr.Markdown(
    #     "## Upload a txt file or Use the Default 'Oppenheimer-movie-wiki.txt' that has already been loaded"
    # )

    # file_name = gr.Textbox(
    #     label="Loaded text file", value=default_text_file, lines=1, interactive=False
    # )
    # upload_button = gr.UploadButton(
    #     label="Click to upload a text file", file_types=["text"], file_count="single"
    # )
    # upload_button.upload(upload_file, upload_button, [file_name, text_file])

    # gr.Markdown("## Enter your question")
    # tokens_slider = gr.Slider(
    #     8,
    #     256,
    #     value=64,
    #     label="Maximum new tokens",
    #     info="A larger `max_new_tokens` parameter value gives you longer text responses but at the cost of a slower response time.",
    # )

    # with gr.Row():
    #     with gr.Column():
    #         print("in column")
    #         ques = gr.Textbox(label="Question", placeholder="Enter text here", lines=3)
    #         print(ques)
    #     with gr.Column():
    #         ans = gr.Textbox(label="Answer", lines=4, interactive=False)
    # with gr.Row():
    #     with gr.Column():
    #         btn = gr.Button("Submit")
    #     with gr.Column():
    #         clear = gr.ClearButton([ques, ans])

    btn.click(fn=generate, inputs=[ques, ans, text_file, 64], outputs=[ans])
    examples = gr.Examples(
        examples=[
            "What is the role of MDA MnS producer in the management function? [3GPP Release 17]",
            "In the plot of the movie, why did Lewis Strauss resent Robert Oppenheimer?",
            "How much money did the Oppenheimer movie make at the US and global box office?",
            "What score did the Oppenheimer movie get on Rotten Tomatoes and Metacritic?",
        ],
        inputs=[ques],
    )

demo.queue().launch()

##test 2

In [ ]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.vectorstores.faiss import FAISS
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings

# Load your model and tokenizer
model_name ="microsoft/phi-2"  # Replace with your desired model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:

# Set up the text generation pipeline
text_gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Set up document loader, vector store, and retriever
document_loader = UnstructuredFileLoader(file_path="/content/23288-i40.txt")
documents = document_loader.load()

# sentence transformers to be used in vector store
# embedding = HuggingFaceEmbeddings()
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/msmarco-distilbert-base-v4",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": False},
)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:

vector_store = FAISS.from_documents(documents, embedding, distance_strategy=DistanceStrategy.COSINE)
retriever = VectorStoreRetriever(vector_store=vector_store)

# Set up retrieval-based QA chain
qa_chain = RetrievalQA(retriever=retriever, model=text_gen_pipeline)

# Define your question
question = "What is the role of MDA MnS producer in the management function? [3GPP Release 17]"

# Retrieve context and generate answer
context = retriever.retrieve(question)
answer = text_gen_pipeline(question, context=context, max_new_tokens=50)[0]['generated_text']

print("Context:", context)
print("Answer:", answer)


NameError: name 'FAISS' is not defined